# Hands-On Assignment 5

In this assignment, you will practice everything that you have learned so far in an end-to-end setting.
You will be provided with a dataset that is **unique to you**, and your task is to perform
all the steps from previous assignments to clean, explore, visualize, and analyze your dataset.

**Written Portion**: Additionally, you will create a report that describes your process and provides insights about your dataset.
Each section that should appear in your report is noted with an orange star (like normal HO tasks).  The report should be  4-6 pages (12 pt font, 1.5 line spacing), and turned in on Canvas as a PDF.

The coding aspect for this assignment will be turned in the same was as all other HO's,
by submitting this file to the autograder.


For this assignment, feel free to make additional functions instead of implementing everything in the provided function.

The objective of this assignment is for you to apply and solidify the skills you have learned in previous assignments.

# Prompt

You have graduated from this class, and are a huge success!
You landed a job doing data science at some fancy company.

You just got a new client with some really interesting problems you get to solve.
Unfortunately, because of a big mess-up on their side the data's metadata got corrupted up
(and the person that used to maintain the data just took a vow of silence and moved to a bog).

The only column you are sure about is the `label` column,
which contains a numeric label for each row.
Aside from that, the client does not know anything about the names, content, or even data types for each column.

Your task is to explore, clean, and analyze this data.
You should have already received an email with the details on obtaining your unique data.
Place it in the same directory as this notebook (and your `grade_local.py` script) and name it `data.txt`.

*I know this prompt may sound unrealistic, but I have literally been in a situation exactly like this.
I was working at a database startup, and one of our clients gave us data with over 70 columns and more than a million records and told us:
"The person who used to manage the data is no longer working with us, but this was the data they used to make all their decisions.
We also lost all the metadata information, like column names."
...
Working in industry is not always glamorous.
-Eriq*

# Part 0: Explore Your Data

Before you start doing things to/with your data, it's always a good idea to load up your data and take a look.

In [1]:
import pandas
import re
import numpy
import sklearn
from scipy import stats
import sklearn.tree
import sklearn.linear_model
import sklearn.neighbors
from sklearn.model_selection import cross_validate
# Modify this to point to your data.
unique_data = pandas.read_csv('lschram_data.txt', sep = "\t")
unique_data

,label,col_00,col_01,col_02,col_03,col_04,col_05,col_06,col_07,col_08,col_09,col_10
0,1,780 m/(s^2),0.8335,santa ana,Basketball,None,1282 N,0.9774,-0.1603,1215,569 m^2,965
1,1,151 m/(s^2),0.8964,Santa Ana,boxing,0.8355,1597 N,0.8088,0.0553,1080,1297 m^2,854
2,4,1254 m/(s^2),1.1378,San Francisco,soccer,0.997,None,0.1076,0.7162,945,581 m^2,671
3,6,692 m/(s^2),0.8615,Long Beach,running,0.6401,756 N,1.2791,0.899,1216,-423 m^2,716
4,2,1012 m/(s^2),0.7492,Riverside,golf,0.9094,1242 N,0.8609,0.8198,1245,817 m^2,300
...,...,...,...,...,...,...,...,...,...,...,...,...
1155,2,342 m/(s^2),1.2886,Sacramento,golf,0.4866,597 N,0.9836,0.6672,886,400 m^2,53
1156,1,1028 m/(s^2),1.264,San Bernardino,basketball,0.7503,1210 N,0.9893,-0.4081,743,602 m^2,903
1157,3,596 m/(s^2),0.6966,Bakersfield,"badminton, baseball",0.8487,820 N,1.1058,0.4713,928,-72 m^2,898
1158,3,1153 m/(s^2),0.8452,Fresno,"BASEBALL, MOTOR SPORTS",0.8279,1125 N,1.0938,0.6802,808,118 m^2,1097


Don't forget to checkout the column information.

In [2]:
unique_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1160 entries, 0 to 1159
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   1160 non-null   int64 
 1   col_00  1152 non-null   object
 2   col_01  1153 non-null   object
 3   col_02  1155 non-null   object
 4   col_03  1153 non-null   object
 5   col_04  1154 non-null   object
 6   col_05  1153 non-null   object
 7   col_06  1148 non-null   object
 8   col_07  1152 non-null   object
 9   col_08  1152 non-null   object
 10  col_09  1152 non-null   object
 11  col_10  1153 non-null   object
dtypes: int64(1), object(11)
memory usage: 108.9+ KB


And any numeric information.

In [3]:
unique_data.describe()

,label
count,1160.000000
mean,3.039655
std,1.984447
min,0.000000
25%,1.000000
50%,3.000000
75%,5.000000
max,6.000000


<h4 style="color: darkorange";>★ Written Task: Introduction</h4>

Briefly describe the dataset you’re given and define the goal of the project and how you approach it.
For example, you can present a basic introduction of your data (shape and proposed data types)
and your goal is to use these features to predict the label of the response variable.
Then you propose a few models that are suitable for this project which will be introduced in the modeling section.

# Part 1: Data Cleaning

As always, we should start with data cleaning.
Take what you learned from HO3 to clean up this messy data to a point where it is ready for machine learning algorithms.

Some things you may want to do:
 - Deal with missing/empty values.
 - Fix numeric columns so that they actually contain numbers.
 - Remove inconsistencies from columns.
 - Assign a data type to each column.

<h4 style="color: darkorange";>★ Task 1.A</h4>

Complete the following function that takes in a DataFrame and outputs a clean version of the DataFrame.
You can assume that the frame has all the same structure as your unique dataset.
You can return the same or a new data frame.

In [4]:
def guess_types(frame):
    for column_name in frame.columns:
        dtype = int
        for row in range(len(frame)):
            value = frame[column_name][row]
            if (isinstance(value, str)):
                dtype = str
                break
            if (isinstance(value, float)):
                dtype = float
        frame[column_name] = frame[column_name].astype(dtype)
    return frame

def drop_sparse_columns(frame, sparsity_threshold):
    for colname in frame:
        bool_arr = pandas.isna(frame[colname])
        if (bool_arr.sum() / len(bool_arr)) > sparsity_threshold:
            frame.pop(colname)
    return frame

def clean_data(frame):
    nan_list = ["None", "NAN", "N/A", "nan", "NaN", "?"]
    for column_name in frame.columns:
        for row_ind in range(len(frame)):
            current_cell = str(frame[column_name][row_ind])
            number_string_list = re.findall(r"-?\d+(?:[,.]\d+)*%?", current_cell)
            if (len(number_string_list) == 0 or (current_cell in nan_list)):
                frame[column_name][row_ind] = numpy.nan
                continue
            first_num = number_string_list[0]
            if ("%" in first_num):
                if ("," in first_num):
                    first_num = (first_num.replace(",", ""))
                frame[column_name][row_ind] = float(first_num[0:-1]) / 100
            elif ("." in first_num):
                frame[column_name][row_ind] = float(first_num)
            elif ("," in first_num):
                frame[column_name][row_ind] = int(first_num.replace(",", ""))
            else:
                frame[column_name][row_ind] = int(first_num)
    frame = guess_types(frame)
    frame = drop_sparse_columns(frame, .5)
    return frame


unique_data = clean_data(unique_data)
unique_data

/tmp/ipykernel_1334/208795403.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frame[column_name][row_ind] = int(first_num)
/tmp/ipykernel_1334/208795403.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frame[column_name][row_ind] = numpy.nan
/tmp/ipykernel_1334/208795403.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frame[column_name][row_ind] = float(first_num)


,label,col_00,col_01,col_04,col_05,col_06,col_07,col_08,col_09,col_10
0,1,780.0,0.8335,NaN,1282.0,0.9774,-0.1603,1215.0,569.0,965.0
1,1,151.0,0.8964,0.8355,1597.0,0.8088,0.0553,1080.0,1297.0,854.0
2,4,1254.0,1.1378,0.9970,NaN,0.1076,0.7162,945.0,581.0,671.0
3,6,692.0,0.8615,0.6401,756.0,1.2791,0.8990,1216.0,-423.0,716.0
4,2,1012.0,0.7492,0.9094,1242.0,0.8609,0.8198,1245.0,817.0,300.0
...,...,...,...,...,...,...,...,...,...,...
1155,2,342.0,1.2886,0.4866,597.0,0.9836,0.6672,886.0,400.0,53.0
1156,1,1028.0,1.2640,0.7503,1210.0,0.9893,-0.4081,743.0,602.0,903.0
1157,3,596.0,0.6966,0.8487,820.0,1.1058,0.4713,928.0,-72.0,898.0
1158,3,1153.0,0.8452,0.8279,1125.0,1.0938,0.6802,808.0,118.0,1097.0


Now we should also be able to view all the numeric columns.

In [5]:
unique_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1160 entries, 0 to 1159
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   label   1160 non-null   int64  
 1   col_00  1147 non-null   float64
 2   col_01  1152 non-null   float64
 3   col_04  1153 non-null   float64
 4   col_05  1146 non-null   float64
 5   col_06  1142 non-null   float64
 6   col_07  1145 non-null   float64
 7   col_08  1148 non-null   float64
 8   col_09  1150 non-null   float64
 9   col_10  1144 non-null   float64
dtypes: float64(9), int64(1)
memory usage: 90.8 KB


In [6]:
unique_data.dropna(inplace=True)

<h4 style="color: darkorange";>★ Written Task: Data Cleaning</h4>

Describe the steps you took for data cleaning.
Why did you do this?
Did you have to make some choices along the way? If so, describe them.

# Part 2: Data Visualization

Once you have cleaned up the data, it is time to explore it and find interesting things.
Part of this exploration, will be visualizing the data in a way that makes it easier for yourself and others to understand.
Use what you have learned in HO1 and HO2 to create some visualizations for your dataset.

<h4 style="color: darkorange";>★ Written Task: Data Visualization</h4>

Create at least two different visualizations that help describe what you see in your dataset.
Include these visualizations in your report along with descriptions of
how you created the visualization,
what data preparation you had to do for the visualization (aside from the data cleaning in the previous part),
and what the visualization tells us about the data.

# Part 3: Modeling

Now that you have a good grasp of your clean data,
it is time to do some machine learning!
(Technically all our previous steps were also machine learning,
but now we get to use classifiers!)

Use the skills you developed to select **three** classifiers and implement them on your data.
For example, you can narrow down your choices to three classifiers which may include:
- Logistic regression
- K-nearest neighbors
- Decision tree
- Or others

<h4 style="color: darkorange";>★ Task 3.A</h4>

Complete the following function that takes in no parameters,
and returns a list with **three** untrained classifiers you are going to explore in this assignment.
This method may set parameters/options for the classifiers, but should not do any training/fitting.

For example, if you wanted to use logistic regression,
then **one** of your list items may be:
```
sklearn.linear_model.LogisticRegression()
```

In [7]:
def create_classifiers():
    regression_model = sklearn.linear_model.LogisticRegression()
    knn_model = sklearn.neighbors.KNeighborsClassifier()
    tree_model = sklearn.tree.DecisionTreeClassifier()
    return [regression_model, knn_model, tree_model]

my_classifiers = create_classifiers()
my_classifiers

[LogisticRegression(), KNeighborsClassifier(), DecisionTreeClassifier()]

Now that we have some classifiers, we can see how they perform.

<h4 style="color: darkorange";>★ Task 3.B</h4>

Complete the following function that takes in an untrained classifier, a DataFrame, and a number of folds.
This function should run k-fold cross validation with the classifier and the data,
and return a list with the accuracy of each run of cross validation.
You can assume that the frame has the column `labels` and the rest of the columns can be considered clean numeric features.

Note that you may have to break your frame into features and labels to do this.
Do not change the passed-in frame (make copies instead).

If you are getting any `ConvergenceWarning`s you may either ignore them,
or try and address them
(they will not affect your autograder score, but may be something to discuss in the written portion of this assignment).

In [8]:
def cross_fold_validation(classifier, frame, folds):
    label = frame['label']
    features = frame.drop('label', axis=1)
    out = cross_validate(classifier, features, label, cv=folds)
    return list(out['test_score'])

my_classifiers_scores = []
for classifier in my_classifiers:
    accuracy_scores = cross_fold_validation(classifier, unique_data, 5)
    my_classifiers_scores.append(accuracy_scores)
    print("Classifier: %s, Accuracy: %s." % (type(classifier).__name__, accuracy_scores))

Classifier: LogisticRegression, Accuracy: [0.6777251184834123, 0.7488151658767772, 0.6872037914691943, 0.6824644549763034, 0.6872037914691943].
Classifier: KNeighborsClassifier, Accuracy: [0.6635071090047393, 0.6777251184834123, 0.6824644549763034, 0.6824644549763034, 0.6966824644549763].
Classifier: DecisionTreeClassifier, Accuracy: [0.8341232227488151, 0.8104265402843602, 0.8436018957345972, 0.8672985781990521, 0.8625592417061612].


/home/lucaa/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/lucaa/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

<h4 style="color: darkorange";>★ Task 3.C</h4>

Complete the following function that takes in two equally-sized lists of numbers and a p-value.
This function should compute whether there is a statistical significance between
these two lists of numbers using a [Student's t-test](https://en.wikipedia.org/wiki/Student%27s_t-test)
at the given p-value.
Return `True` if there is a statistical significance, and `False` otherwise.
Hint: If you wish, you may use the `ttest_ind()` [method](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html) provided in the scipy package. 

In [9]:
def significance_test(a_values, b_values, p_value):
    return stats.ttest_ind(a_values, b_values)[1] < p_value

for i in range(len(my_classifiers)):
    for j in range(i + 1, len(my_classifiers)):
        significant = significance_test(my_classifiers_scores[i], my_classifiers_scores[j], 0.10)
        print("%s vs %s: %s" % (type(my_classifiers[i]).__name__,
                                type(my_classifiers[j]).__name__, significant))

LogisticRegression vs KNeighborsClassifier: False
LogisticRegression vs DecisionTreeClassifier: True
KNeighborsClassifier vs DecisionTreeClassifier: True


<h4 style="color: darkorange";>★ Written Task: Modeling</h4>

Describe the classifiers you have chosen.
Be sure to include all details about any parameter settings used for the algorithms.

Compare the performance of your models using k-fold validation.
You may look at accuracy, F1 or other measures.

Then, briefly summarize your results.
Are your results statistically significant?
Is there a clear winner?
What do the standard deviations look like, and what do they tell us about the different models?
Include a table like Table 1.

<center>Table 1: Every table need a caption.</center>

| Model | Mean Accuracy | Standard Deviation of Accuracy |
|-------|---------------|--------------------------------|
| Logistic Regression | 0.724 | 0.004
| K-Nearest Neighbor | 0.750 | 0.003
| Decision Tree | 0.655 | 0.011

# Part 4: Analysis

Now, take some time to go over your results for each classifier and try to make sense of them.
 - Why do some classifiers work better than others?
 - Would another evaluation metric work better than vanilla accuracy?
 - Is there still a problem in the data that should fixed in data cleaning?
 - Does the statistical significance between the different classifiers make sense?
 - Are there parameters for the classifier that I can tweak to get better performance?

<h4 style="color: darkorange";>★ Written Task: Analysis</h4>

Discuss your observations, the relationship you found, and how you applied concepts from the class to this project.
For example, you may find that some feature has the most impact in predicting your response variable or removing a feature improves the model accuracy.
Or you may observe that your training accuracy is much higher than your test accuracy and you may want to explain what issues may arise.

# Part 5: Conclusion

<h4 style="color: darkorange";>★ Written Task: Conclusion</h4>

Briefly summarize the important results and conclusions presented in the project.
What are the important points illustrated by your work?
Are there any areas for further investigation or improvement?

<h4 style="color: darkorange";>★ Written Task: References</h4>

Include a standard bibliography with citations referring to techniques or published papers you used throughout your report (if you used any).

For example:
```
[1] Derpanopoulos, G. (n.d.). Bayesian Model Checking & Comparison.
https://georgederpa.github.io/teaching/modelChecking.html.
```